# Pipeline 1: Business Search with Named Entity Recognition

## Overview
This notebook implements a Haystack pipeline that processes natural language queries to search for businesses on Yelp. The pipeline uses Named Entity Recognition (NER) to extract locations and keywords from user queries, then executes a search using the Yelp Business Search API.

## What This Pipeline Does
1. Takes a natural language query (e.g., "best restaurants in town for Mexican food")
2. Extracts entities (locations, keywords) using NER
3. Constructs a search query for the Yelp API
4. Returns JSON results with business IDs, aliases, and other details

## Use Cases
- Natural language business search
- Location and keyword extraction
- Multi-business discovery
- Data gathering for downstream pipelines

## Pipeline Architecture
```
User Query → NER Extraction → Keyword Extraction → Yelp API Search → JSON Results
```

## API Reference
- Endpoint: SEARCH BUSINESS
- URL: https://yelp-business-reviews.p.rapidapi.com/search
- Required params: location, query
- Returns: business_id, alias, name, rating, categories, etc.

## Setup and Environment Variables

Before running this notebook:
1. Sign up for RapidAPI and subscribe to the Yelp Business Reviews API
2. Create a `.env` file in the project root with your API key:
   ```
   RAPID_API_KEY=your_key_here
   OPENAI_API_KEY=your_openai_key_here
   ```

In [1]:
# Import required libraries
import requests
from dotenv import load_dotenv
import os
from haystack import Pipeline, component, Document
from haystack.components.extractors import NamedEntityExtractor
from typing import List, Dict, Any
import json
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Load environment variables
load_dotenv(".env")
RAPID_API_KEY = os.getenv("RAPID_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("✓ Environment variables loaded successfully")

✓ Environment variables loaded successfully


## Custom Component 1: Query to Document Converter

This component converts a user's natural language query into a Haystack Document object that can be processed by the NER extractor.

In [2]:
@component
class QueryToDocument:
    """
    Converts a natural language query string into a Haystack Document.
    This allows the query to be processed by NER and other document-based components.
    
    Input:
        - query (str): Natural language query from user
    
    Output:
        - documents (List[Document]): Single document containing the query
    """
    
    def __init__(self):
        """Initialize the component with a logger."""
        self.logger = logging.getLogger(__name__ + ".QueryToDocument")
    
    @component.output_types(documents=List[Document])
    def run(self, query: str) -> Dict[str, List[Document]]:
        """
        Convert query string to Document object.
        
        Args:
            query: User's natural language query
            
        Returns:
            Dictionary with 'documents' key containing the query as a Document
        """
        self.logger.info(f"Converting query to document: '{query}'")
        doc = Document(content=query)
        self.logger.debug(f"Created document with ID: {doc.id}")
        return {"documents": [doc]}


print("✓ QueryToDocument component defined")

✓ QueryToDocument component defined


## Custom Component 2: Entity Keyword Extractor

This component processes documents with NER entities and extracts:
- **Location (LOC)**: Geographic locations for the Yelp search
- **Keywords**: Other entities and original query terms for the search query

It intelligently combines entities from NER with the original query to build comprehensive search parameters.

In [3]:
@component
class EntityKeywordExtractor:
    """
    Extracts location and keywords from documents with named entities.
    
    This component processes documents that have been annotated by Haystack's 
    NamedEntityExtractor. It reads NamedEntityAnnotation objects from the 
    document metadata and extracts location and keyword information.
    
    This component:
    1. Identifies location entities (LOC) from NER results
    2. Collects other entities (ORG, MISC) as keywords
    3. Combines with original query terms
    4. Returns structured location and keyword data
    
    Input:
        - documents (List[Document]): Documents with named_entities in metadata
          (output from NamedEntityExtractor)
    
    Output:
        - location (str): Extracted location for Yelp search
        - keywords (List[str]): Search keywords
        - original_query (str): Original user query
    """
    
    def __init__(self):
        """Initialize the component with a logger."""
        self.logger = logging.getLogger(__name__ + ".EntityKeywordExtractor")
    
    @component.output_types(
        location=str,
        keywords=List[str],
        original_query=str
    )
    def run(self, documents: List[Document]) -> Dict[str, Any]:
        """
        Extract location and keywords from NER-processed documents.
        
        This method processes NamedEntityAnnotation objects stored in the 
        document's metadata by Haystack's NamedEntityExtractor.
        
        Args:
            documents: List of documents with named_entities metadata
            
        Returns:
            Dictionary containing location, keywords, and original query
        """
        if not documents:
            self.logger.warning("No documents provided for entity extraction")
            return {
                "location": "",
                "keywords": [],
                "original_query": ""
            }
        
        doc = documents[0]
        original_query = doc.content
        self.logger.info(f"Processing query: '{original_query}'")
        
        # Get NamedEntityAnnotation objects from document metadata
        # These are added by Haystack's NamedEntityExtractor
        named_entities = doc.meta.get("named_entities", [])
        self.logger.debug(f"Found {len(named_entities)} named entities")
        
        locations = []
        keywords = []
        
        # Process each NamedEntityAnnotation
        for annotation in named_entities:
            # Extract the entity text from the document content using start/end indices
            entity_text = doc.content[annotation.start:annotation.end]
            entity_label = annotation.entity
            confidence = annotation.score
            
            self.logger.debug(f"Entity: '{entity_text}' - Type: {entity_label} - Confidence: {confidence:.3f}")
            
            # Only use high-confidence entities (>0.7)
            if confidence > 0.7:
                # Check if it's a location entity
                if entity_label == "LOC":
                    locations.append(entity_text)
                    self.logger.info(f"Extracted location: '{entity_text}'")
                # Use organizations and miscellaneous entities as keywords
                elif entity_label in ["ORG", "MISC"]:
                    keywords.append(entity_text)
                    self.logger.info(f"Extracted keyword: '{entity_text}' ({entity_label})")
        
        # Use first location found, or default to empty
        location = locations[0] if locations else ""
        if not location:
            self.logger.warning("No location extracted from entities")
        
        # Combine keywords with query terms (remove common stop words)
        stop_words = ["the", "in", "for", "a", "an", "best", "good", "great", "town"]
        query_words = [word for word in original_query.split() 
                      if word.lower() not in stop_words]
        
        # Merge and deduplicate keywords
        all_keywords = list(set(keywords + query_words))
        
        self.logger.info(f"Final extraction - Location: '{location}', Keywords: {all_keywords}")
        
        return {
            "location": location,
            "keywords": all_keywords,
            "original_query": original_query
        }

print("✓ EntityKeywordExtractor component defined")

✓ EntityKeywordExtractor component defined


## Custom Component 3: Yelp Business Search

This component executes the actual Yelp API search using the extracted location and keywords.
It constructs the API request and returns the business results as JSON.

In [4]:
@component
class YelpBusinessSearch:
    """
    Searches for businesses using the Yelp Business Reviews API.
    
    This component:
    1. Accepts location and keywords from previous component
    2. Constructs API request with proper headers and parameters
    3. Executes the search via RapidAPI
    4. Returns parsed JSON results with business information
    
    Input:
        - location (str): Geographic location to search
        - keywords (List[str]): Search keywords
        - original_query (str): Original user query
    
    Output:
        - results (Dict): JSON response from Yelp API
        - search_params (Dict): The parameters used for the search
    """
    
    def __init__(self, api_key: str):
        """
        Initialize the Yelp search component.
        
        Args:
            api_key: RapidAPI key for Yelp Business Reviews API
        """
        self.api_key = api_key
        self.base_url = "https://yelp-business-reviews.p.rapidapi.com/search"
        self.headers = {
            "x-rapidapi-key": self.api_key,
            "x-rapidapi-host": "yelp-business-reviews.p.rapidapi.com"
        }
        self.logger = logging.getLogger(__name__ + ".YelpBusinessSearch")
    
    @component.output_types(
        results=Dict,
        search_params=Dict
    )
    def run(self, location: str, keywords: List[str], original_query: str) -> Dict[str, Any]:
        """
        Execute Yelp business search.
        
        Args:
            location: Location to search in
            keywords: List of search keywords
            original_query: Original user query
            
        Returns:
            Dictionary with API results and search parameters
        """
        # If no location extracted, try to use a default or query
        if not location:
            location = "United States"  # Default fallback
            self.logger.warning(f"No location provided, using fallback: '{location}'")
        
        # Construct query string from keywords
        query_string = " ".join(keywords) if keywords else original_query
        
        # Build query parameters
        querystring = {
            "location": location,
            "query": query_string
        }
        
        self.logger.info(f"Searching Yelp API - Location: '{location}', Query: '{query_string}'")
        
        try:
            # Execute API request
            response = requests.get(
                self.base_url,
                headers=self.headers,
                params=querystring
            )
            
            self.logger.debug(f"API response status: {response.status_code}")
            
            # Parse JSON response
            results = response.json()
            
            result_count = results.get('resultCount', 0)
            self.logger.info(f"Successfully retrieved {result_count} businesses")
            
            return {
                "results": results,
                "search_params": {
                    "location": location,
                    "query": query_string,
                    "original_query": original_query
                }
            }
            
        except Exception as e:
            self.logger.error(f"Error during Yelp API search: {str(e)}")
            return {
                "results": {"error": str(e), "resultCount": 0, "results": []},
                "search_params": {
                    "location": location,
                    "query": query_string,
                    "original_query": original_query
                }
            }

print("✓ YelpBusinessSearch component defined")

✓ YelpBusinessSearch component defined


## Build the Pipeline

Now we'll assemble all components into a complete pipeline:
1. QueryToDocument - Convert query to document
2. NamedEntityExtractor - Extract entities using NER
3. EntityKeywordExtractor - Extract location and keywords
4. YelpBusinessSearch - Search Yelp API

The pipeline connects these components in sequence to process the user query end-to-end.

In [5]:
# Initialize pipeline
pipeline = Pipeline()

# Initialize components
query_converter = QueryToDocument()
ner_extractor = NamedEntityExtractor(backend="hugging_face", model="dslim/bert-base-NER")
keyword_extractor = EntityKeywordExtractor()
yelp_search = YelpBusinessSearch(api_key=RAPID_API_KEY)

# Warm up the NER model (load into memory)
print("Loading NER model...")
ner_extractor.warm_up()
print("✓ NER model loaded")

# Add components to pipeline
pipeline.add_component("query_converter", query_converter)
pipeline.add_component("ner_extractor", ner_extractor)
pipeline.add_component("keyword_extractor", keyword_extractor)
pipeline.add_component("yelp_search", yelp_search)

# Connect components
pipeline.connect("query_converter.documents", "ner_extractor.documents")
pipeline.connect("ner_extractor.documents", "keyword_extractor.documents")
pipeline.connect("keyword_extractor.location", "yelp_search.location")
pipeline.connect("keyword_extractor.keywords", "yelp_search.keywords")
pipeline.connect("keyword_extractor.original_query", "yelp_search.original_query")

print("✓ Pipeline built successfully")
print("\nPipeline structure:")
print("Query → QueryToDocument → NER → KeywordExtractor → YelpSearch → Results")

Loading NER model...


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps
Device set to use mps


✓ NER model loaded
✓ Pipeline built successfully

Pipeline structure:
Query → QueryToDocument → NER → KeywordExtractor → YelpSearch → Results


## Test the Pipeline

Let's test the pipeline with various natural language queries to see how it extracts entities and searches for businesses.

In [6]:
# Test query 1: Restaurant search with explicit location
test_query = "best Mexican restaurants in Madison, WI"

print(f"Testing query: '{test_query}'")
print("="*60)

result = pipeline.run(data={"query_converter": {"query": test_query}})

# Display search parameters
print("\nExtracted Search Parameters:")
print(f"Location: {result['yelp_search']['search_params']['location']}")
print(f"Query: {result['yelp_search']['search_params']['query']}")
print(f"Original: {result['yelp_search']['search_params']['original_query']}")

# Display results
api_results = result['yelp_search']['results']
print(f"\n✓ Found {api_results.get('resultCount', 0)} businesses")

if 'results' in api_results and len(api_results['results']) > 0:
    print("\nTop 5 Results:")
    for i, business in enumerate(api_results['results'][:5], 1):
        print(f"\n{i}. {business['name']}")
        print(f"   Business ID: {business['bizId']}")
        print(f"   Alias: {business['alias']}")
        print(f"   Rating: {business['rating']} ({business['reviewCount']} reviews)")
        print(f"   Categories: {', '.join(business['categories'])}")
        print(f"   Price: {business.get('priceRange', 'N/A')}")

2025-11-07 15:34:01,005 - haystack.core.pipeline.base - INFO - Warming up component ner_extractor...
2025-11-07 15:34:01,077 - haystack.core.pipeline.pipeline - INFO - Running component query_converter
2025-11-07 15:34:01,078 - __main__.QueryToDocument - INFO - Converting query to document: 'best Mexican restaurants in Madison, WI'
2025-11-07 15:34:01,078 - haystack.core.pipeline.pipeline - INFO - Running component ner_extractor
2025-11-07 15:34:01,077 - haystack.core.pipeline.pipeline - INFO - Running component query_converter
2025-11-07 15:34:01,078 - __main__.QueryToDocument - INFO - Converting query to document: 'best Mexican restaurants in Madison, WI'
2025-11-07 15:34:01,078 - haystack.core.pipeline.pipeline - INFO - Running component ner_extractor
2025-11-07 15:34:01,192 - haystack.core.pipeline.pipeline - INFO - Running component keyword_extractor
2025-11-07 15:34:01,192 - __main__.EntityKeywordExtractor - INFO - Processing query: 'best Mexican restaurants in Madison, WI'
2025-

Testing query: 'best Mexican restaurants in Madison, WI'


2025-11-07 15:34:04,987 - __main__.YelpBusinessSearch - INFO - Successfully retrieved 50 businesses



Extracted Search Parameters:
Location: Madison
Query: WI restaurants Mexican Madison,
Original: best Mexican restaurants in Madison, WI

✓ Found 50 businesses

Top 5 Results:

1. Garibaldi Mexican Restaurant
   Business ID: ue9ahXtW39ZbnfS8b2XIAQ
   Alias: garibaldi-mexican-restaurant-madison
   Rating: 4.3 (101 reviews)
   Categories: Mexican
   Price: None

2. Taqueria Guadalajara
   Business ID: S3-JpB5dBhMXmauMqeoN-g
   Alias: taqueria-guadalajara-madison
   Rating: 4.2 (490 reviews)
   Categories: Mexican
   Price: $$

3. A La Brasa Mexican Grill
   Business ID: uYoSb2rNi77LMQZ-Nn1Epg
   Alias: a-la-brasa-mexican-grill-madison-2
   Rating: 4.1 (158 reviews)
   Categories: Mexican
   Price: $

4. Mercado Marimar
   Business ID: R6cznooObGDV-_DU-uZViQ
   Alias: mercado-marimar-madison-2
   Rating: 4.5 (56 reviews)
   Categories: Supermarkets, Mexican
   Price: $

5. El Rancho Mexican Grill
   Business ID: E3zvCzX7gquT9srY4pUwLg
   Alias: el-rancho-mexican-grill-madison
   Rating: 4

In [7]:
# Test query 2: Coffee shop search
test_query = "coffee shops in San Francisco"

print(f"Testing query: '{test_query}'")
print("="*60)

result = pipeline.run(data={"query_converter": {"query": test_query}})

# Display search parameters
print("\nExtracted Search Parameters:")
print(f"Location: {result['yelp_search']['search_params']['location']}")
print(f"Query: {result['yelp_search']['search_params']['query']}")

# Display results
api_results = result['yelp_search']['results']
print(f"\n✓ Found {api_results.get('resultCount', 0)} businesses")

if 'results' in api_results and len(api_results['results']) > 0:
    print("\nTop 3 Results:")
    for i, business in enumerate(api_results['results'][:3], 1):
        print(f"\n{i}. {business['name']}")
        print(f"   Business ID: {business['bizId']}")
        print(f"   Alias: {business['alias']}")
        print(f"   Rating: {business['rating']}")

2025-11-07 15:34:05,004 - haystack.core.pipeline.base - INFO - Warming up component ner_extractor...
2025-11-07 15:34:05,005 - haystack.core.pipeline.pipeline - INFO - Running component query_converter
2025-11-07 15:34:05,006 - __main__.QueryToDocument - INFO - Converting query to document: 'coffee shops in San Francisco'
2025-11-07 15:34:05,006 - haystack.core.pipeline.pipeline - INFO - Running component ner_extractor
2025-11-07 15:34:05,005 - haystack.core.pipeline.pipeline - INFO - Running component query_converter
2025-11-07 15:34:05,006 - __main__.QueryToDocument - INFO - Converting query to document: 'coffee shops in San Francisco'
2025-11-07 15:34:05,006 - haystack.core.pipeline.pipeline - INFO - Running component ner_extractor
2025-11-07 15:34:05,130 - haystack.core.pipeline.pipeline - INFO - Running component keyword_extractor
2025-11-07 15:34:05,131 - __main__.EntityKeywordExtractor - INFO - Processing query: 'coffee shops in San Francisco'
2025-11-07 15:34:05,131 - __main__.

Testing query: 'coffee shops in San Francisco'


2025-11-07 15:34:09,139 - __main__.YelpBusinessSearch - INFO - Successfully retrieved 240 businesses



Extracted Search Parameters:
Location: San Francisco
Query: Francisco San shops coffee

✓ Found 240 businesses

Top 3 Results:

1. Q Specialty Coffee
   Business ID: UXCtOlad97FzZNWyxOii3g
   Alias: q-specialty-coffee-san-francisco
   Rating: 4.7

2. Oishii Matcha
   Business ID: q7_Haz8CBJo-YAYRA70lPQ
   Alias: oishii-matcha-san-francisco
   Rating: 4.1

3. Heyma Yemeni Coffee
   Business ID: nS8oKy7iyxgOmh2cg-qj6g
   Alias: heyma-yemeni-coffee-san-francisco
   Rating: 4.6


In [8]:
# Test query 3: Generic food search (no specific location)
test_query = "Italian pizza places"

print(f"Testing query: '{test_query}'")
print("="*60)

result = pipeline.run(data={"query_converter": {"query": test_query}})

# Display search parameters
print("\nExtracted Search Parameters:")
print(f"Location: {result['yelp_search']['search_params']['location']}")
print(f"Query: {result['yelp_search']['search_params']['query']}")

# Display results
api_results = result['yelp_search']['results']
print(f"\n✓ Found {api_results.get('resultCount', 0)} businesses")

2025-11-07 15:34:09,144 - haystack.core.pipeline.base - INFO - Warming up component ner_extractor...
2025-11-07 15:34:09,145 - haystack.core.pipeline.pipeline - INFO - Running component query_converter
2025-11-07 15:34:09,145 - __main__.QueryToDocument - INFO - Converting query to document: 'Italian pizza places'
2025-11-07 15:34:09,145 - haystack.core.pipeline.pipeline - INFO - Running component ner_extractor
2025-11-07 15:34:09,145 - haystack.core.pipeline.pipeline - INFO - Running component query_converter
2025-11-07 15:34:09,145 - __main__.QueryToDocument - INFO - Converting query to document: 'Italian pizza places'
2025-11-07 15:34:09,145 - haystack.core.pipeline.pipeline - INFO - Running component ner_extractor
2025-11-07 15:34:09,233 - haystack.core.pipeline.pipeline - INFO - Running component keyword_extractor
2025-11-07 15:34:09,233 - __main__.EntityKeywordExtractor - INFO - Processing query: 'Italian pizza places'
2025-11-07 15:34:09,233 - __main__.EntityKeywordExtractor - IN

Testing query: 'Italian pizza places'


2025-11-07 15:34:13,942 - __main__.YelpBusinessSearch - INFO - Successfully retrieved 240 businesses



Extracted Search Parameters:
Location: United States
Query: Italian places pizza

✓ Found 240 businesses


In [21]:
result

{'yelp_search': {'results': {'resultCount': 11,
   'currentPage': 1,
   'totalPages': 2,
   'location': {'city': 'Madison'},
   'results': [{'bizId': 'Xp_cWXY5rxDLkX-wqUg-iQ',
     'name': "Brennan's Market",
     'alias': 'brennans-market-madison-3',
     'serviceArea': None,
     'lat': 43.0556119,
     'lon': -89.524489,
     'rating': 4.5,
     'reviewCount': 81,
     'categories': ['Off Licence', 'Greengrocers', 'Cheese Shops'],
     'services': [],
     'businessHighlights': [],
     'priceRange': '$$',
     'phone': '(608) 833-2893',
     'website': 'https://www.brennansmarket.com',
     'images': ['https://s3-media0.fl.yelpcdn.com/bphoto/WuL0iPar3ea_lGrMInAPzw/348s.jpg']},
    {'bizId': 'RlulW-HxTuUVk8wSspNXIw',
     'name': 'Wisconsin Cheese Mart',
     'alias': 'wisconsin-cheese-mart-madison',
     'serviceArea': None,
     'lat': 43.074266,
     'lon': -89.3873044,
     'rating': 4.2,
     'reviewCount': 25,
     'categories': ['Cheese Shops'],
     'services': [],
     'bus

## Helper Function: Extract Business IDs

This utility function extracts business IDs from the API results for use in downstream pipelines (Pipeline 2 and 3).

In [11]:
def extract_business_info(api_results: Dict, max_results: int = 5) -> List[str]:
    """
    Extract business IDs from Yelp API results.
    
    Args:
        api_results: JSON response from Yelp search API
        max_results: Maximum number of businesses to extract (default: 5)
    
    Returns:
        List of business IDs
    """
    business_ids = []
    
    if 'results' in api_results:
        for business in api_results['results'][:max_results]:
            business_ids.append(business['bizId'])
    
    return business_ids

# Test the helper function
result = pipeline.run(data={"query_converter": {"query": "cheese curds in Madison, WI"}})
api_results = result['yelp_search']['results']

business_info = extract_business_info(api_results, max_results=3)


2025-11-07 15:34:52,516 - haystack.core.pipeline.base - INFO - Warming up component ner_extractor...
2025-11-07 15:34:52,517 - haystack.core.pipeline.pipeline - INFO - Running component query_converter
2025-11-07 15:34:52,517 - __main__.QueryToDocument - INFO - Converting query to document: 'cheese curds in Madison, WI'
2025-11-07 15:34:52,518 - haystack.core.pipeline.pipeline - INFO - Running component ner_extractor
2025-11-07 15:34:52,517 - haystack.core.pipeline.pipeline - INFO - Running component query_converter
2025-11-07 15:34:52,517 - __main__.QueryToDocument - INFO - Converting query to document: 'cheese curds in Madison, WI'
2025-11-07 15:34:52,518 - haystack.core.pipeline.pipeline - INFO - Running component ner_extractor
2025-11-07 15:34:52,621 - haystack.core.pipeline.pipeline - INFO - Running component keyword_extractor
2025-11-07 15:34:52,622 - __main__.EntityKeywordExtractor - INFO - Processing query: 'cheese curds in Madison, WI'
2025-11-07 15:34:52,622 - __main__.Entity

In [15]:
business_ids = extract_business_info(api_results, max_results=3)
print("Extracted Business IDs:")
print(f"Business IDs: {business_ids}")

Extracted Business IDs:
Business IDs: ['Xp_cWXY5rxDLkX-wqUg-iQ', 'RlulW-HxTuUVk8wSspNXIw', 'OnKyFU0FsiPVd8Jwl6st6g']


## Summary

### What We Built
- **Pipeline 1** successfully extracts entities from natural language queries and searches Yelp
- The pipeline returns JSON results with business IDs and aliases needed for downstream processing

### Key Outputs
- `business_id` (bizId): Unique identifier for each business
- `business_alias`: URL-friendly business identifier
- Full business data including rating, location, categories, etc.

### Next Steps
These outputs feed into:
- **Pipeline 2**: Get detailed business information and website content
- **Pipeline 3**: Fetch and analyze business reviews

### Usage Example
```python
# Run the pipeline
result = pipeline.run(data={"query_converter": {"query": "your query here"}})

# Extract business IDs for downstream use
api_results = result['yelp_search']['results']
business_ids = extract_business_info(api_results, max_results=5)
```